In [79]:
#!pip install t4

In [80]:
import numpy as np 
import pandas as pd 

import keras
from keras.models import Sequential
from keras.layers import Input, Dense, Dropout, Flatten
from keras.optimizers import Adam, Nadam, Adamax

#import t4

In [11]:
data = pd.read_csv('Data/Top10_Artikel_wolf.csv')
data = data[data.DatumZeit >= '2017-04']

data['DatumZeit'] = pd.to_datetime(data['DatumZeit'])
data["DayofWeek"] = data['DatumZeit'].dt.dayofweek

mean_on = ['LinkKasse', 'DayofWeek', 'LinkArtikel']

for i in range(1, 1 + 3):    
    data['lag'+str(i)] = data.groupby(by = mean_on )['Anzahl'].shift(i)
    
TestData = '2019-06-01'
data = data.dropna()
train_size = data[data.DatumZeit <= TestData].shape[0]
train, test = data[:train_size], data[train_size:]

In [70]:
clf = Sequential()

clf.add( Dense(64, activation='relu', input_shape=(3,) ) )
clf.add(Dropout(0.05))
clf.add( Dense(64, activation='relu') )
clf.add(Dropout(0.05))
clf.add(Dense(1, activation='relu'))


def root_mean_squared_error(y_true, y_pred):
        return (keras.backend.mean(keras.backend.square(y_pred - y_true)))

clf.compile( optimizer=Adamax(lr=1e-1), loss = root_mean_squared_error )

In [73]:
drop_nn_columns = ['DatumZeit', 'LinkKasse', 'LinkArtikel', 'Anzahl', 'DayofWeek']
history = clf.fit(
           train.drop(columns=drop_nn_columns), train.Anzahl, 
    epochs=5, batch_size=109,
    validation_data=( test.drop(columns=drop_nn_columns) , test.Anzahl),
    verbose=True
        )

Train on 7600 samples, validate on 1920 samples
Epoch 1/5
7600/7600 [==============================] - 0s 25us/step - loss: 2382.2774 - val_loss: 1578.9704
Epoch 2/5
7600/7600 [==============================] - 0s 24us/step - loss: 2325.3328 - val_loss: 1422.3992
Epoch 3/5
7600/7600 [==============================] - 0s 24us/step - loss: 2324.2123 - val_loss: 1291.0923
Epoch 4/5
7600/7600 [==============================] - 0s 24us/step - loss: 2184.8806 - val_loss: 1484.6046
Epoch 5/5
7600/7600 [==============================] - 0s 24us/step - loss: 2280.4733 - val_loss: 1518.9631


In [74]:
clf.predict( np.array(test.drop(columns=drop_nn_columns)) )

array([[832.71387 ],
       [562.7478  ],
       [ 97.14497 ],
       ...,
       [ 11.671348],
       [ 17.244797],
       [ 12.983937]], dtype=float32)

In [76]:
clf.save('clf.h5')

In [83]:
'''(t4.Package()
     .set("clf.h5")
     #.push("quilt/quilt_sagemaker_demo", "s3://quilt-example", registry="s3://quilt-example")
)'''

(local Package)
 └─clf.h5